# 데이터 준비

In [3]:
import os
import pandas as pd

# 'data/raw_data' 경로 설정
root_dir = 'data/'
visitor_city = pd.read_csv(os.path.join(root_dir, "city_of_festival_visitors.csv"))  # 무주축제 방문객 top 18 지역들 (시군구명,od_cnts,시도명,행정동코드,위도,경도)
address = pd.read_csv(os.path.join(root_dir, "address_with_lon_lat_final.csv"))  # 행정동코드 + 위도경도 (행정동코드,시도명,시군구명,읍면동명,동리명,위도,경도)
mooju = set(list(address[address['시군구명'] == '무주군']['행정동코드']))  # 무주군 행정동코드
other_city = list(address.merge(visitor_city, on=['시도명', '시군구명'])['행정동코드_x'])  # 다른 지역들 행정동코드 모음

In [4]:
address['시도 시군구'] = address['시도명'].fillna('') + ' ' + address['시군구명'].fillna('')

# 양쪽 값이 모두 null인 경우 빈 문자열 처리
address['시도 시군구'] = address['시도 시군구'].str.strip()
address.head()

,행정동코드,시도명,시군구명,읍면동명,동리명,위도,경도,시도 시군구
0,1100000000,서울특별시,NaN,NaN,서울특별시,37.566679,126.978291,서울특별시
1,1111000000,서울특별시,종로구,NaN,종로구,37.580695,126.982799,서울특별시 종로구
2,1111051500,서울특별시,종로구,청운효자동,세종로,37.579997,126.976930,서울특별시 종로구
3,1111051500,서울특별시,종로구,청운효자동,옥인동,37.583480,126.963850,서울특별시 종로구
4,1111051500,서울특별시,종로구,청운효자동,누하동,37.578998,126.967561,서울특별시 종로구


# 함수

### 준비

In [5]:
# 시도시군구명으로 위경도 좌표를 반환하는 함수
def get_coordinates(lst):
    coordinates = []
    for name in lst:
        try:
            # '시도 시군구'로 '위도', '경도' 가져옴
            target = address[address['시도 시군구'] == name][['위도','경도']].iloc[0]
            x, y = target.iloc[0], target.iloc[1]
            coordinates.append((x,y))
        except IndexError:
            # 해당 '시도 시군구'에 대한 데이터가 없는 경우
            print(f"{name} 지역의 위도, 경도 정보가 없습니다.")
    return coordinates

In [6]:
import yaml
import requests

# api 사용을 위한 key값 가져오기 
with open("config.yaml", "r") as file:
    config = yaml.safe_load(file)
naver_api_id = config['naver api']['id']
naver_api_key = config['naver api']['key']
kakao_api_key = config['kakao api']['key']

# 네이버 api 요청 함수
def naver_request(start, goal):
    # 요청
    url = 'https://naveropenapi.apigw.ntruss.com/map-direction/v1/driving'
    params = {
        'goal': f'{goal[1]},{goal[0]}',
        'start': f'{start[1]},{start[0]}',
    }
    headers = {
        'x-ncp-apigw-api-key-id': naver_api_id,
        'x-ncp-apigw-api-key': naver_api_key
    }
    response = requests.get(url, headers=headers, params=params)

    # 응답
    data = response.json()
    if data['code'] == 0:
        if data['route']['traoptimal']:
            summary = data['route']['traoptimal'][0]['summary']
            distance = summary['distance']
            duration = summary['duration']
            return distance, duration
        else:
            print("응답 에러: 'traoptimal' 데이터가 없습니다.")
            return -1, -1
    else:
        print("요청 실패: ", data['message'])
        return -1, -1

# 카카오 api 요청 함수
def kakao_request(start, goal):
    # 요청
    url = 'https://apis-navi.kakaomobility.com/v1/directions'
    params = {
        'origin': f'{start[1]},{start[0]}',
        'destination': f'{goal[1]},{goal[0]}',
        'alternatives': True,
    }
    headers = {
        'Authorization': f'KakaoAK {kakao_api_key}'
    }
    response = requests.get(url, headers=headers, params=params)

    # 응답
    if response.status_code == 200:
        data = response.json()
        if data['routes']:
            summary = data['routes'][0]['summary']
            distance = summary.get('distance')
            duration = summary.get('duration')
            return distance, duration
        else:
            print("응답 에러: 'routes' 데이터가 없습니다.")
            return -1, -1
    else:
        print("요청 실패")
        return -1, -1

In [7]:
# 노드 간 이동시간, 이동거리 테이블 반환하는 함수
def get_path_info(coordinates, request_func):
    n = len(coordinates)
    distance_table = [[0] * n for _ in range(n)]
    duration_table = [[0] * n for _ in range(n)]

    for i in range(n):
        for j in range(n):
            if j > i:
                start = coordinates[i]
                goal = coordinates[j]
                distance, duration = request_func(start, goal)
                distance_table[i][j], distance_table[j][i] = distance, distance
                duration_table[i][j], duration_table[j][i] = duration, duration

    return distance_table, duration_table

### 탐색

In [8]:
import numpy as np
import networkx as nx

# 모든 경로 찾기
def find_all_routes(G, start, target, visited=None, path=None):
    if visited is None:
        visited = set()
    if path is None:
        path = []

    visited.add(start)
    path.append(start)

    if start == target:
        yield path.copy()
    else:
        for neighbor in G.neighbors(start):
            if neighbor not in visited:
                yield from find_all_routes(G, neighbor, target, visited, path)

    path.pop()
    visited.remove(start)

# 최적 경로 탐색 함수
def find_optimal_routes(G, stations, min_threshold, max_threshold, step):
    optimal_routes = []
    max_stops = 0
    explored_routes = set()  # 탐색된 경로를 저장할 집합

    # 여러 배수 값을 탐색
    for threshold in np.arange(min_threshold, max_threshold + step, step):

        for station in stations.keys():
            if station != "전라북도 무주군":  # 목적지가 아니면
                routes = list(find_all_routes(G, station, "전라북도 무주군"))

                for route in routes:
                    valid_route = True
                    total_distance = 0
                    total_time = 0
                    total_weight = 0  # 가중치 누적

                    # 각 경로의 모든 구간에 대해 확인
                    for i in range(len(route) - 1):
                        current_station = route[i]

                        # 현재 노드에서 무주군으로 바로 이동할 때의 거리와 시간
                        if "전라북도 무주군" in G[current_station]:  # 직접 경로가 있는 경우에만
                            direct_distance = G[current_station]["전라북도 무주군"]['distance']
                            direct_time = G[current_station]["전라북도 무주군"]['time']

                            # 현재 구간의 거리와 시간을 누적
                            total_distance += G[route[i]][route[i + 1]]['distance']
                            total_time += G[route[i]][route[i + 1]]['time']

                            # 각 중간 노드에서도 주어진 배수(factor) 조건 확인
                            if total_distance > direct_distance * threshold or total_time > direct_time * threshold:
                                valid_route = False
                                break

                        # 가중치 누적 (출발지 제외)
                        if i > 0:
                            total_weight += stations[current_station]

                    if valid_route:
                        # 경로를 집합에 저장할 수 있는 형태로 변환 (정렬하여 순서 상관없이 비교 하기 위함)
                        route_tuple = tuple(sorted(route))

                        # 만약 해당 경로가 이전 배수에서 이미 탐색되었다면 제외
                        if route_tuple not in explored_routes:
                            explored_routes.add(route_tuple)  # 새로운 경로로 저장

                            # 최대 정거장 수 확인
                            if len(route) > max_stops:
                                max_stops = len(route)
                                optimal_routes = [(route, total_distance, total_time, total_weight, threshold)]
                            elif len(route) == max_stops:
                                optimal_routes.append((route, total_distance, total_time, total_weight, threshold))

    return optimal_routes



### 가중치

In [9]:
def get_weights(nodes):
    stations = {
        "세종특별자치시": 1,
        "대전광역시 유성구": 2,  # 가중치 2로 증가
        "대전광역시 서구": 2,    # 가중치 2로 증가
        "대전광역시 대덕구": 1,
        "대전광역시 중구": 1,
        "충청남도 금산군": 1,
        "충청북도 영동군": 2,    # 가중치 2로 증가
        "전라북도 무주군": 0,    # 목적지
    }


    return stations

# 출력

In [ ]:
def print_optimal_routes(optimal_routes, name):
    print(f"{name}지역 최적의 셔틀 노선")
    if optimal_routes:
        for route, total_distance, total_time, total_weight, threshold in optimal_routes:
            # 총 거리를 km로 변환하고 소수점 아래 두째 자리까지 포맷
            total_distance_km = total_distance / 1000  # 미터를 킬로미터로 변환
            total_distance_formatted = f"{total_distance_km:.2f} km"

            # 총 시간을 시간과 분으로 변환 (반올림)
            total_time_hours = total_time // 60
            total_time_minutes = round(total_time % 60)

            print(f"최적의 셔틀 노선: {route} | 총 거리: {total_distance_formatted} | 총 시간: {total_time_hours}시간 {total_time_minutes}분 | 총 가중치: {total_weight} | threshold: {threshold:.2f}")
    else:
        print("유효한 경로가 없습니다.")

---

# 메인

In [10]:
def search(nodes, name, stations):
    # 1. 위경도 좌표 기반으로 경유지 간 이동거리, 이동시간 테이블 만듦

    # 경유지 위경도 좌표 
    coordinates = get_coordinates(nodes) 

    # 네이버 api
    # n_distance_table, n_duration_table = get_path_info(coordinates, naver_request)
    # n_duration_min_table = [[round(value / 3600, 2) for value in row] for row in n_duration_table]

    # 카카오 api
    k_distance_table, k_duration_table = get_path_info(coordinates, kakao_request)
    k_duration_min_table = [[round(value / 60, 2) for value in row] for row in k_duration_table]


    distance_table = k_distance_table
    duration_table = k_duration_min_table

    # 2. 정거장 후보지와 가중치
    # stations = get_weights(nodes)  

    # 3. 탐색
    # 그래프 생성
    G = nx.DiGraph()

    # 그래프에 엣지 추가
    for i, from_station in enumerate(stations.keys()):
        for j, to_station in enumerate(stations.keys()):
            if i != j:
                G.add_edge(from_station, to_station, 
                        distance=distance_table[i][j], 
                        time=duration_table[i][j])
                
    # 탐색 범위 설정
    min_threshold = 1.1
    max_threshold = 1.5
    step = 0.01

    # 최적 경로 탐색 실행
    optimal_routes = find_optimal_routes(G, stations, min_threshold, max_threshold, step)

    # 결과 출력
    print_optimal_routes(optimal_routes, name)


In [11]:
# 경유지 리스트
daejun = ['세종특별자치시', '대전광역시 유성구', '대전광역시 서구', '대전광역시 대덕구', '대전광역시 중구', '충청남도 금산군', '충청북도 영동군', '전라북도 무주군']
jeonbuk = ['전라북도 군산시', '전라북도 익산시', '전라북도 전주시 완산구', '전라북도 전주시 덕진구', '전라북도 진안군', '전라북도 장수군', '전라북도 무주군']
stations_daejun = {
        "세종특별자치시": 1,
        "대전광역시 유성구": 2,  # 가중치 2로 증가
        "대전광역시 서구": 2,    # 가중치 2로 증가
        "대전광역시 대덕구": 1,
        "대전광역시 중구": 1,
        "충청남도 금산군": 1,
        "충청북도 영동군": 2,    # 가중치 2로 증가
        "전라북도 무주군": 0,    # 목적지
    }
stations_jeonbuk = {
    '전라북도 군산시': 1, 
    '전라북도 익산시': 1, 
    '전라북도 전주시 완산구': 2, 
    '전라북도 전주시 덕진구': 2, 
    '전라북도 진안군': 1, 
    '전라북도 장수군': 2,
    '전라북도 무주군': 0,    # 목적지
}

In [12]:
search(daejun, "대전", stations_daejun)

대전지역 최적의 셔틀 노선
최적의 셔틀 노선: ['대전광역시 유성구', '대전광역시 서구', '대전광역시 대덕구', '전라북도 무주군'] | 총 거리: 70926 | 총 시간: 82.00999999999999 | 배수: 1.50
최적의 셔틀 노선: ['대전광역시 서구', '대전광역시 대덕구', '대전광역시 중구', '전라북도 무주군'] | 총 거리: 61778 | 총 시간: 84.6 | 배수: 1.50
최적의 셔틀 노선: ['대전광역시 중구', '대전광역시 대덕구', '대전광역시 유성구', '전라북도 무주군'] | 총 거리: 83096 | 총 시간: 91.07 | 배수: 1.50


In [14]:
search(jeonbuk, "전북", stations_jeonbuk)

전북지역 최적의 셔틀 노선
최적의 셔틀 노선: ['전라북도 전주시 완산구', '전라북도 전주시 덕진구', '전라북도 무주군'] | 총 거리: 75432 | 총 시간: 98.69999999999999 | 배수: 1.30
최적의 셔틀 노선: ['전라북도 군산시', '전라북도 익산시', '전라북도 무주군'] | 총 거리: 118161 | 총 시간: 127.83000000000001 | 배수: 1.40
최적의 셔틀 노선: ['전라북도 익산시', '전라북도 전주시 덕진구', '전라북도 무주군'] | 총 거리: 93818 | 총 시간: 120.02 | 배수: 1.50
최적의 셔틀 노선: ['전라북도 전주시 완산구', '전라북도 익산시', '전라북도 무주군'] | 총 거리: 126795 | 총 시간: 140.0 | 배수: 1.50
최적의 셔틀 노선: ['전라북도 전주시 덕진구', '전라북도 군산시', '전라북도 무주군'] | 총 거리: 153420 | 총 시간: 150.68 | 배수: 1.50
